# Necessary Libraries and Settings

In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


# Loading Function

In [2]:
def load():
    data = pd.read_csv(".....")
    return data


# Outliers

## Grafik

In [3]:
# sns.boxplot(x=df[col], whis = 1.5) 
# plt.show(block = True)

 ## Fonksiyonlar

In [4]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

# outlier_thresholds(df, "Age")
# outlier_thresholds(df, "Fare")

In [5]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False
    
# check_outlier(df, "Age")
# check_outlier(df, "Fare")

In [6]:
def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index

# Col Type Analysis

In [7]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

# cat_cols, num_cols, cat_but_car = grab_col_names(df)
# num_cols = [col for col in num_cols if col not in "PassengerId"]

# Handling Outliers

In [8]:
def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers

# for col in num_cols:
#     new_df = remove_outlier(df, col)

In [9]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
# for col in num_cols:
#     print(col, check_outlier(df, col))

# for col in num_cols:
#     replace_with_thresholds(df, col)

# for col in num_cols:
#     print(col, check_outlier(df, col))

## Usage Examples

In [10]:
# for col in num_cols:
#     print(col, check_outlier(df, col))



# outlier_thresholds(df, "Age")
# check_outlier(df, "Age")
# grab_outliers(df, "Age", index=True)

# remove_outlier(df, "Age").shape
# replace_with_thresholds(df, "Age")
# check_outlier(df, "Age")

# Local Outlier Factor

In [11]:
# df = df.select_dtypes(include=['float64', 'int64'])
# df = df.dropna()

# low, up = outlier_thresholds(df, "carat")

# df[((df["carat"] < low) | (df["carat"] > up))].shape

# low, up = outlier_thresholds(df, "depth")

# df[((df["depth"] < low) | (df["depth"] > up))].shape

# clf = LocalOutlierFactor(n_neighbors=20)
# clf.fit_predict(df)

# df_scores = clf.negative_outlier_factor_
# df_scores[0:5]
# # df_scores = -df_scores
# np.sort(df_scores)[0:5]

# scores = pd.DataFrame(np.sort(df_scores))
# scores.plot(stacked=True, xlim=[0, 50], style='.-')
# plt.show(block=True)

# th = np.sort(df_scores)[3]

# df[df_scores < th] 

# Missing Values

## Missing value graphs

In [12]:
# msno.bar(df)
# plt.show(block=True)

# msno.matrix(df) #değişkenlerdeki eksikliklerin birlikte çıkıp çıkmadığı ile alakalı bilgi veriyor.
# plt.show(block=True)

# msno.heatmap(df)
# plt.show(block=True)

In [13]:
# # degiskenlerdeki eksik deger sayisi
# df.isnull().sum()

# # degiskenlerdeki tam deger sayisi
# df.notnull().sum()

# # veri setindeki toplam eksik deger sayisi
# df.isnull().sum().sum()

# # en az bir tane eksik degere sahip olan gözlem birimleri
# df[df.isnull().any(axis=1)]

# # tam olan gözlem birimleri
# df[df.notnull().all(axis=1)]

# # Azalan şekilde sıralamak
# df.isnull().sum().sort_values(ascending=False)

# (df.isnull().sum() / df.shape[0] * 100).sort_values(ascending=False) # Yüzdesel olarak gösterip sıralar

# na_cols = [col for col in df.columns if df[col].isnull().sum() > 0] # na olan sutunları verir


In [14]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns
# missing_values_table(df, True)

In [15]:
def missing_vs_target(dataframe, target, na_columns):
    temp_df = dataframe.copy()

    for col in na_columns:
        temp_df[col + '_NA_FLAG'] = np.where(temp_df[col].isnull(), 1, 0)

    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_NA_")].columns

    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN": temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")
        
        
        
# na_cols = missing_values_table(df, True)
# missing_vs_target(df, "Survived", na_cols)

# Handling Missing Values

In [16]:
# df["Age"].fillna(df["Age"].mean())
# df["Age"].fillna(df["Age"].median())
# df["Age"].fillna(0).isnull()

# df.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0).head() # üsttekilerin genelleştirilmiş hali

# df["Embarked"].fillna(df["Embarked"].mode()[0]) # Kategorik değişkende eksik var ise en çok tekrar eden değeri ile doldurabiliriz.

# df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis=0) #üsttekinin genelleştirilmiş hali


# df["Age"].fillna(df.groupby("Sex")["Age"].transform("mean")).isnull().sum() # kateogrik kırılımda grupların kendı ortalaması ıle doldurma.


# df.loc[(df["Age"].isnull()) & (df["Sex"]=="female"), "Age"] = df.groupby("Sex")["Age"].mean()["female"] # yukardakı islemın acıklaması

# df.loc[(df["Age"].isnull()) & (df["Sex"]=="male"), "Age"] = df.groupby("Sex")["Age"].mean()["male"] # yukardakı islemın acıklaması


## KNN method, extraction needed from the code below

In [17]:
# cat_cols, num_cols, cat_but_car = grab_col_names(df)
# num_cols = [col for col in num_cols if col not in "PassengerId"]
# dff = pd.get_dummies(df[cat_cols + num_cols], drop_first=True)

# dff.head()

# # değişkenlerin standartlatırılması
# scaler = MinMaxScaler()
# dff = pd.DataFrame(scaler.fit_transform(dff), columns=dff.columns)
# dff.head()


# # knn'in uygulanması.
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=5) ################# en yakın komşuların 5 tanesinin ortalamasını atar diyor
#                                     # en yakından kastı eksik olan değişkenin dışındaki değişkenlerin benzerliğinden mi bahsediyor.
#                                     # burada age değişkenı dışında diğer değişkenlerde de missing value olsa idi hepsini dolurcak mıydı?
# dff = pd.DataFrame(imputer.fit_transform(dff), columns=dff.columns)
# dff.head()

# dff = pd.DataFrame(scaler.inverse_transform(dff), columns=dff.columns)

# df["age_imputed_knn"] = dff[["Age"]]

# df.loc[df["Age"].isnull(), ["Age", "age_imputed_knn"]]
# df.loc[df["Age"].isnull()]

# Encoding (Label Encoding, One-Hot Encoding, Rare Encoding)

In [18]:
# le.inverse_transform([0, 1]) # label değerlerinin karşılıklarını verir.

def label_encoder(dataframe, binary_col): # bu fonksiyon çalışırken değişkende eksik değer var ise ona 2 değerini veriyor.
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col]) #dönüşüm yaparken alfabetik sıraya göre 0 dan başlayarak labellar ve iki değişkenli olduğu için birine 0 diğerine 1 verir.
    return dataframe
# binary_cols = [col for col in df.columns if df[col].dtype not in ["int64", "float64"] and df[col].nunique() == 2]
# for col in binary_cols:
#     label_encoder(df, col)

In [19]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=True, dummy_na =False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first, dummy_na=dummy_na)
    return dataframe
# ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]
# one_hot_encoder(df, ohe_cols).head()

# Rare Encoding

In [20]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()
        
        
# for col in cat_cols:
#     cat_summary(df, col)

In [21]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show(block=True)
        
# for col in age_cols:
#     num_summary(df, col, plot=True)

In [22]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

# rare_analyser(df, "TARGET", cat_cols)

In [23]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy() #copy yapmadan eşitleme yapsaydık temp_dfteki değişiklikler dataframe i etkileyecek miydi?

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df


# new_df = rare_encoder(df, 0.01)

# rare_analyser(new_df, "TARGET", cat_cols)

# Feature Scaling (Özellik Ölçeklendirme)

In [24]:
# scaler = StandardScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])

# rs = RobustScaler()
# df[num_cols] = rs.fit_transform(df[num_cols])


# mms = MinMaxScaler()
# df[num_cols] = mms.fit_transform(df[[num_cols]])






# Feature Extraction

#### Examples from Titanic Dataset

In [25]:
# df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype('int') # değişkenin boş olan değerlerine 0 dolu olan değerlerine 1 veriyor böylece değişkenin dolu olup olmamasının bağımlı değişkene olan etkisine bakıyoruz. Değerin ne olduğundan bağımsız
# df.groupby("NEW_CABIN_BOOL").agg({"Survived": "mean"})


# from statsmodels.stats.proportion import proportions_ztest

# test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
#                                              df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],

#                                       nobs=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
#                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])

# print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue)) 
# # Sonrasında da ab testing yaparak gercekten bir fark var mı yok mu ona bakıyoruz.

In [26]:
# df.loc[((df['SibSp'] + df['Parch']) > 0), "NEW_IS_ALONE"] = "NO"
# df.loc[((df['SibSp'] + df['Parch']) == 0), "NEW_IS_ALONE"] = "YES"

# test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].sum(),
#                                              df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].sum()],

#                                       nobs=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],
#                                             df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].shape[0]])

# print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

### Wordcount

In [27]:
# # Letter Count
# ###################

# df["NEW_NAME_COUNT"] = df["Name"].str.len()

# ###################
# # Word Count
# ###################

# df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))

#### Extraction status(special letters) from name variable

In [28]:
# df['NEW_TITLE'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# df[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived": "mean", "Age": ["count", "mean"]})

#### Date examples

In [29]:
# dff['Timestamp'] = pd.to_datetime(dff["Timestamp"], format="%Y-%m-%d")

# # year
# dff['year'] = dff['Timestamp'].dt.year

# # month
# dff['month'] = dff['Timestamp'].dt.month

# # year diff
# dff['year_diff'] = date.today().year - dff['Timestamp'].dt.year

# # month diff (iki tarih arasındaki ay farkı): yıl farkı + ay farkı
# dff['month_diff'] = (date.today().year - dff['Timestamp'].dt.year) * 12 + date.today().month - dff['Timestamp'].dt.month


# # day name
# dff['day_name'] = dff['Timestamp'].dt.day_name()

### Examples of extraction new variables from numeric variables titanic dataset

In [30]:
# df["NEW_AGE_PCLASS"] = df["Age"] * df["Pclass"] # bunun duzgun çalışması için yaş değişkenin standtartlaşmaya tabi tutulması gerekiyor imiş

# df["NEW_FAMILY_SIZE"] = df["SibSp"] + df["Parch"] + 1

# df.loc[(df['SEX'] == 'male') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'

# df.loc[(df['SEX'] == 'male') & (df['Age'] > 21) & (df['Age'] < 50), 'NEW_SEX_CAT'] = 'maturemale'

# df.loc[(df['SEX'] == 'male') & (df['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniormale'

# df.loc[(df['SEX'] == 'female') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'

# df.loc[(df['SEX'] == 'female') & (df['Age'] > 21) & (df['Age'] < 50), 'NEW_SEX_CAT'] = 'maturefemale'

# df.loc[(df['SEX'] == 'female') & (df['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniorfemale'


# Relation of independent variables with dependent variable

In [31]:
def plot_importance(model, features, save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                      ascending=False)[0:len(X)])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')


# plot_importance(rf_model, X_train)